# Solving the Traveling Salesman  Problem using Local Search

Points: 10

## The [Traveling Salesman Problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem)

* __Goal:__ Find the shortest tour visiting each of $n$ cities exactly once and returning back to the starting city. Given are pairwise distances between cities, where $d_{i,j}$ is the distance from city $i$ to city $j$. 

* __State space:__ Each state represents a tour. The cities are numbered and a tour can be expressed as vector  $\pi$ with the order in which the cities are visited (a [permutation](https://en.wikipedia.org/wiki/Permutation)). That is, $\pi(1)$ is the index of the first city to visit, $\pi(2)$ the index of the second, and so on.

* __Objective function:__ Minimize the tour length. The optimization problem is to find the optimal tour $\pi^*$ through the $n$ cities and returning to the starting city:

  > minimize: $\mathrm{tourlength}(\pi) = d_{\pi(n),\pi(1)} + \sum_{i = 1}^{n-1} d_{\pi(i),\pi(i+1)}$
  > 
  > subject to: $\pi \ \text{is a valid permutation vector}$

* __Local moves:__ Exchange two cities in the order.

## Helper functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

np.set_printoptions(precision=2)
pd.set_option('display.precision', 2)

# make the results repeatable
np.random.seed(1234)

In [ ]:
def random_tour(n):
    """Create a random tour"""
    
    tour = list(range(n))
    random.shuffle(tour)
    return(tour)

random_tour(10)

In [ ]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

def random_tsp(n):
    """
    Create a random (Euclidean) traveling salesman problem. Choose n points randomly in a 1 x 1 unit square and calulates a 
    pairwise Euclidean distance matrix.
    """
    
    pos = pd.DataFrame({
        "x" : np.random.uniform(size = n),
        "y" : np.random.uniform(size = n)
    })
    
    dist = squareform(pdist(pos))
    
    return({"pos": pos, "dist": dist})
    
tsp = random_tsp(10)

print(f"Positions:\n{tsp['pos']}")
print(f"Distance matrix:\n{pd.DataFrame(tsp['dist'])})")

In [ ]:
def tour_length(tsp, tour):
    """Caclulate the length of a tour, i.e., the objective function."""
    
    # make sure tour is a Python list (not an array or a numpy.array)
    if not isinstance(tour, list): tour = tour.tolist()
    
    tl = 0
    dist = tsp["dist"]
    
    for i in range(len(tour)-1):
        tl += dist[tour[i], tour[i+1]]
    
    tl += dist[tour[-1], tour[0]]
    
    return(tl)
        
tour = random_tour(10)
tour_length(tsp, tour)

In [ ]:
def show_tsp(tsp, tour=None):  
    """display the traveling salesman problem and a tour."""
    
    pos = tsp["pos"]
    
    plt.scatter(pos["x"], pos["y"])
    
    if tour is not None:
        # make sure tour is a Python list (not an array or a numpy.array)
        if not isinstance(tour, list): 
            tour = tour.tolist()
        
        print(f"Tour length: {round(tour_length(tsp, tour), 2)}")
        
        pos_ = pos.reindex(tour)
        pos_ = pd.concat([pos_, pos_.head(1)])  # Sửa ở đây
        plt.plot(pos_["x"], pos_["y"])
    
    plt.show()
    
show_tsp(tsp, tour)

## Use R to find a solution

Load rpy2, make sure the R [TSP package](https://CRAN.R-project.org/package=TSP) is installed and prepare the distance matrix.

In [ ]:
%load_ext rpy2.ipython

%R if(!"TSP" %in% rownames(installed.packages())) install.packages("TSP", repos="http://cran.us.r-project.org")
%R if(!"microbenchmark" %in% rownames(installed.packages())) install.packages("microbenchmark", repos="http://cran.us.r-project.org")

d = tsp["dist"]

Solve the TSP using [`solve_TSP`](https://www.rdocumentation.org/packages/TSP/versions/1.1-10/topics/solve_TSP) with the default heuristic. Note that 2-opt is steepest ascend hill climbing with exchanging two cities. `rep=100` means 100 random restarts.

In [ ]:
%%R -i d -o tour

library("TSP")

tsp <- TSP(d)
print(tsp)

tour <- solve_TSP(tsp, rep = 100)
print(tour)

# R starts index with 1, but Python starts at 0
tour <- tour - 1L

In [ ]:
show_tsp(tsp, tour)

How long does it take to solve the problem 100 times?

In [ ]:
%%R -i d

library("microbenchmark")

microbenchmark(tsp <- TSP(d))

## Steepest-ascend Hill Climbing Search [3 Points]

Calculate the objective function for all local moves (move each queen within its column) and always choose the best among all local moves.

In [ ]:
# Code goes here

## Steepest-ascend Hill Climbing Search with Random Restarts [1 Point]

Steepest-ascend with random restarts.

In [ ]:
# Code goes here

## Stochastic Hill Climbing [1 Points]

Chooses randomly from among all uphill moves.

In [ ]:
# Code goes here

## First-choice Hill Climbing [1 Point]

First-choice hill climbing is a type of stochastic hill climbing that generates one random local neighbor at a time and accept it if it has a better objective function value than the current state.

In [9]:
def first_choice_hill_climbing(tsp, max_iter=1000):
    tour = random_tour(len(tsp['dist']))
    current_length = tour_length(tsp, tour)
    for _ in range(max_iter):
        improved = False
        neighbors = []
        for i in range(len(tour)):
            for j in range(i+1, len(tour)):
                neighbor = tour[:]
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbors.append(neighbor)
        random.shuffle(neighbors)
        for neighbor in neighbors:
            neighbor_length = tour_length(tsp, neighbor)
            if neighbor_length < current_length:
                tour = neighbor
                current_length = neighbor_length
                improved = True
                break
        if not improved:
            break
    return tour, current_length

## Simulated Annealing [2 Points]

In [10]:
def simulated_annealing(tsp, T0=1000, alpha=0.99, T_min=1e-5, max_steps=10000):
    tour = random_tour(len(tsp['dist']))
    current_length = tour_length(tsp, tour)
    T = T0
    for t in range(max_steps):
        if T < T_min:
            break
        i, j = random.sample(range(len(tour)), 2)
        neighbor = tour[:]
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        neighbor_length = tour_length(tsp, neighbor)
        delta = neighbor_length - current_length
        if delta < 0 or random.random() < math.exp(-delta / T):
            tour = neighbor
            current_length = neighbor_length
        T *= alpha
    return tour, current_length

## Compare Performance [2 Points]

Use runtime, scalability (number of cities), and best objective function value to compare the algorithms on boards of different sizes.  

For timing you can use the `time` package.

In [11]:
import time

t0 = time.time()
print("Do something")
t1 = time.time()

print(f"This took: {(t1-t0) * 1e3} milliseconds")

Do something
This took: 0.0 milliseconds


In [13]:
import time
sizes = [10, 20, 30, 40, 50, 60, 70, 80, 90 , 100]
algorithms = {
    'First-Choice HC': first_choice_hill_climbing,
    'Simulated Annealing': simulated_annealing
}
results = {}
for n in sizes:
    tsp = random_tsp(n)
    results[n] = {}
    for name, func in algorithms.items():
        start = time.time()
        _, length = func(tsp)
        runtime = time.time() - start
        results[n][name] = {'runtime': runtime, 'length': length}
print(results)

{10: {'First-Choice HC': {'runtime': 0.0009996891021728516, 'length': np.float64(2.9275752417834022)}, 'Simulated Annealing': {'runtime': 0.006998777389526367, 'length': np.float64(3.3296064980615845)}}, 20: {'First-Choice HC': {'runtime': 0.008000850677490234, 'length': np.float64(4.231729423267925)}, 'Simulated Annealing': {'runtime': 0.012000560760498047, 'length': np.float64(4.174479691231499)}}, 30: {'First-Choice HC': {'runtime': 0.013004302978515625, 'length': np.float64(6.678896176403101)}, 'Simulated Annealing': {'runtime': 0.013000011444091797, 'length': np.float64(6.831243221873798)}}, 40: {'First-Choice HC': {'runtime': 0.10600018501281738, 'length': np.float64(7.848853917395781)}, 'Simulated Annealing': {'runtime': 0.015000104904174805, 'length': np.float64(8.634205961463884)}}, 50: {'First-Choice HC': {'runtime': 0.1150050163269043, 'length': np.float64(8.367348740866733)}, 'Simulated Annealing': {'runtime': 0.018000364303588867, 'length': np.float64(11.622200110437706)}}

## Bonus: Genetic Algorithm [+1 Point]

In [ ]:
# Code goes here